In [1]:
!pip install ultralytics roboflow

from ultralytics import YOLO
import os
from pathlib import Path
import yaml
import shutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 3.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yol

In [2]:
# ============================================================================
# MONTAR GOOGLE DRIVE
# ============================================================================

from google.colab import drive
drive.mount('/content/drive')

DRIVE_RESULTS_PATH = '/content/drive/MyDrive/modelo_treinado_imgs_reais_aug'
os.makedirs(DRIVE_RESULTS_PATH, exist_ok=True)

print(f"✓ Google Drive montado!")
print(f"✓ Resultados serão salvos em: {DRIVE_RESULTS_PATH}")

Mounted at /content/drive
✓ Google Drive montado!
✓ Resultados serão salvos em: /content/drive/MyDrive/modelo_treinado_imgs_reais_aug


In [3]:
from roboflow import Roboflow

ROBOFLOW_API_KEY = "cN0yG1huh8mBnMJ4TUHv"
WORKSPACE_NAME = "tcc-hhkda"
PROJECT_NAME = "imgs_reais_aug-uyjyn"
VERSION = 1

print("Conectando ao Roboflow...")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(WORKSPACE_NAME).project(PROJECT_NAME)

print(f"\nBaixando dataset '{PROJECT_NAME}' versão {VERSION}...")
dataset = project.version(VERSION).download("yolov8")

DATASET_PATH = dataset.location
print(f"\n✓ Dataset baixado em: {DATASET_PATH}")

print("\nEstrutura do dataset:")
print(f"  - Train: {os.path.join(DATASET_PATH, 'train')}")
print(f"  - Valid: {os.path.join(DATASET_PATH, 'valid')}")
print(f"  - Test: {os.path.join(DATASET_PATH, 'test')}")

Conectando ao Roboflow...
loading Roboflow workspace...
loading Roboflow project...

Baixando dataset 'imgs_reais_aug-uyjyn' versão 1...



Extracting Dataset Version Zip to imgs_reais_aug-1 in yolov8:: 100%|██████████| 4347/4347 [00:00<00:00, 4946.26it/s]


✓ Dataset baixado em: /content/imgs_reais_aug-1

Estrutura do dataset:
  - Train: /content/imgs_reais_aug-1/train
  - Valid: /content/imgs_reais_aug-1/valid
  - Test: /content/imgs_reais_aug-1/test


In [ ]:
# ============================================================================
# CRIAR/VERIFICAR ARQUIVO data.yaml
# ============================================================================

data_yaml_content = f"""
path: {DATASET_PATH}
train: train/images
val: valid/images

names:
  0: machine
  1: worker
"""

yaml_path = os.path.join(DATASET_PATH, 'data.yaml')
with open(yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"Arquivo data.yaml criado em: {yaml_path}")

Arquivo data.yaml criado em: /content/imgs_reais_aug-1/data.yaml


In [ ]:
# ============================================================================
# VERIFICAÇÃO DO DATASET
# ============================================================================

def verificar_dataset(dataset_path):
    print("\n=== Verificando Dataset ===")

    train_images = os.path.join(dataset_path, 'valid', 'images')
    train_labels = os.path.join(dataset_path, 'valid', 'labels')
    valid_images = os.path.join(dataset_path, 'train', 'images')
    valid_labels = os.path.join(dataset_path, 'train', 'labels')

    paths = [train_images, train_labels, valid_images, valid_labels]

    for path in paths:
        if os.path.exists(path):
            count = len(os.listdir(path))
            print(f"✓ {path}: {count} arquivos")
        else:
            print(f"✗ {path}: NÃO ENCONTRADO")

    print("\n")

verificar_dataset(DATASET_PATH)


=== Verificando Dataset ===
✓ /content/imgs_reais_aug-1/valid/images: 400 arquivos
✓ /content/imgs_reais_aug-1/valid/labels: 400 arquivos
✓ /content/imgs_reais_aug-1/train/images: 1769 arquivos
✓ /content/imgs_reais_aug-1/train/labels: 1769 arquivos




In [6]:
# ============================================================================
# CONFIGURAÇÃO DO TREINAMENTO
# ============================================================================

EPOCHS = 200
BATCH_SIZE = 16
IMAGE_SIZE = 1280
MODEL_SIZE = 'l'
PATIENCE = 50
DEVICE = 0

RESULTS_DIR = '/content/runs/tcc'
os.makedirs(RESULTS_DIR, exist_ok=True)

model = YOLO(f'yolov8{MODEL_SIZE}.pt')

print(f"\nModelo YOLOv8{MODEL_SIZE} carregado com sucesso!")


Modelo YOLOv8l carregado com sucesso!


In [ ]:
# ============================================================================
# TREINAMENTO
# ============================================================================

print("\n=== Iniciando Treinamento ===\n")

results = model.train(
    data=yaml_path,
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    name='canteiro_obras_detection',
    patience=PATIENCE,
    save=True,
    device=DEVICE,
    workers=2,
    project=RESULTS_DIR,
    exist_ok=True,
    pretrained=True,
    optimizer='auto',
    verbose=True,
    seed=42,
)

print("\n=== Treinamento Concluído ===\n")

# ============================================================================
# SALVAR NO GOOGLE DRIVE
# ============================================================================

print("\n=== Salvando Resultados no Google Drive ===\n")

training_results_path = os.path.join(RESULTS_DIR, 'canteiro_obras_detection')

from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M")
dynamic_drive_path = os.path.join(DRIVE_RESULTS_PATH, timestamp)

if os.path.exists(training_results_path):
    if os.path.exists(dynamic_drive_path):
        shutil.rmtree(dynamic_drive_path)

    shutil.copytree(training_results_path, dynamic_drive_path)
    print(f"✓ Resultados completos salvos em:\n  {dynamic_drive_path}")

    params_file_path = os.path.join(dynamic_drive_path, 'training_parameters.txt')
    with open(params_file_path, 'w') as f:
        f.write(f"Epochs: {EPOCHS}\n")
        f.write(f"Batch Size: {BATCH_SIZE}\n")
        f.write(f"Image Size: {IMAGE_SIZE}\n")
        f.write(f"Model Size: {MODEL_SIZE}\n")
        f.write(f"Patience: {PATIENCE}\n")
        f.write(f"Device: {DEVICE}\n")
        f.write(f"Dataset Path: {DATASET_PATH}\n")
        f.write(f"Results Directory: {RESULTS_DIR}\n")
        f.write(f"Optimizer: auto\n")
        f.write(f"Seed: 42\n")

    print(f"\n✓ Parâmetros do treinamento salvos em:\n  {params_file_path}")

    best_model_source = os.path.join(training_results_path, 'weights', 'best.pt')
    best_model_drive = os.path.join(dynamic_drive_path, 'best_model.pt')
    shutil.copy(best_model_source, best_model_drive)
    print(f"\n✓ Melhor modelo salvo em:\n  {best_model_drive}")

    last_model_source = os.path.join(training_results_path, 'weights', 'last.pt')
    last_model_drive = os.path.join(dynamic_drive_path, 'last_model.pt')
    shutil.copy(last_model_source, last_model_drive)
    print(f"\n✓ Último modelo salvo em:\n  {last_model_drive}")
else:
    print("Pasta de resultados não encontrada!")


=== Iniciando Treinamento ===

Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/imgs_reais_aug-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=canteiro_obras_detection, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto,